In [254]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import glob

In [255]:
def plot_grayscale_image(image,title="Image"):
    plt.imshow(image,'gray')
    plt.title(title)
    plt.show()

In [256]:
def plot_color_image(image,title="Image"):
    plt.imshow(image)
    plt.title(title)
    plt.show()

### Q1. Search the web for Colour Halftoning algorithms. Select one of them and write a detailed report on it OR implement the selected algorithm and show results on the test images.

Implementing Random dot color dithering

### Q2. Try coming up with your own error diffusion coefficients and implement the standard error-diffusion algorithm. Compare the performance of your coefficients against Floyd-Steinberg's on this image. Discuss the patterns visible in yours and in Floyd-Steinberg's at the various gray levels.

Floyd Steinberg's Algorithm

In [257]:
def error_diffusion_algo(image, indexes, num_of_indexes, coeffs, result_image):
    new_image = image.copy()
    for i in range(1,new_image.shape[0]-2):
        for j in range(1,new_image.shape[1]-2):
            error = 0
            if new_image[i][j] < 128 : # threshold for binarisation - 128
                error = new_image[i][j]
                new_image[i][j] = 0                
            else:
                error = new_image[i][j] - 255
                new_image[i][j] = 255
            for k in range(num_of_indexes):
                new_image[i+indexes[k][0]][j+indexes[k][1]] = np.clip(
                                                                (new_image[i+indexes[k][0]][j+indexes[k][1]]\
                                                                + coeffs[k] * error), 0, 255)    
#     new_image = np.uint8(new_image)
    cv.imwrite(result_image, new_image)
    return new_image

In [258]:
image = cv.imread('Test_data/ed-eg.png',0)

floyd_steinberg_indexes = [[0,1], [1,1], [1,0], [1,-1]]
floyd_steinberg_coeffs = [7 / 16, 1 / 16, 5 / 16, 3 / 16]
fd_num_of_indexes = 4
error_diffusion_algo(image, floyd_steinberg_indexes, fd_num_of_indexes, floyd_steinberg_coeffs, "Q2.Results/FloydSteinbergDiffused.png")

my_diffusion_indexes = [[0,1], [1,1], [1,0],[0,2], [2,2], [2,0]]
qw1my_num_of_indexes = 6
my_diffused_image = error_diffusion_algo(image, my_diffusion_indexes, my_num_of_indexes, my_diffusion_coeffs, "Q2.Results/MyDiffused.png")

### Q3. 
### (a).Implement an algorithm to simulate the grayscale output from a colour filter array. The function prototype is image colour_filter (image, filter).That is, it takes an input colour image and a colour filter as parameters and returns a grayscale image. 
### (b). Implement a demosaicking algorithm with the prototype image demosaic (image, filter).The input image is a grayscale image output by the colour_filter algorithm and the corresponding filter array; the output is a colour image.


a. Apply color filter array on color image to get grayscale image.

In [265]:
def color_to_grayscale(image, color_filter, result_image):
    neighbors = [[0,0], [0,1], [1,0], [1,1]]
    new_image = np.zeros((image.shape[0],image.shape[1]))
    # pass 2x2 filter over the image, for each filter component value, 
    # retain that component value in pixel
    for i in range(0, image.shape[0]-2, 2):
        for j in range(0, image.shape[1]-2, 2):
            for k in range(0, 4):
                new_image[i+neighbors[k][0]][j+neighbors[k][1]] = image[i+k][j+k][color_filter[k]]
    cv.imwrite(result_image, new_image)

b. Demosaicking algorithm - apply color filter array on grayscale image - convert to color image

In [270]:
color_filter1 = [1, 2, 0, 1] # Bayer's mask which Canon uses
color_filter2 = [2, 1, 1, 0] # Bayer's mask which other manufaturers use
'''
[G,B]
[R,G]
'''
image = cv.imread('Test_data/orange-flower.ppm')
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
color_filter_2grayscale(image, color_filter1,'Q3.Grayscale_image_results/filter1_orange_flower.jpg')
color_filter_2grayscale(image, color_filter2,'Q3.Grayscale_image_results/filter2_orange_flower.jpg')

image = cv.imread('Test_data/fall-colours.jpg')
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
color_filter_2grayscale(image, color_filter1,'Q3.Grayscale_image_results/filter1_fall_colors.jpg')
color_filter_2grayscale(image, color_filter2,'Q3.Grayscale_image_results/filter2_fall_colors.jpg')

image = cv.imread('Test_data/waterplane.ppm')
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
color_filter_2grayscale(image, color_filter1,'Q3.Grayscale_image_results/filter1_waterplane.jpg')
color_filter_2grayscale(image, color_filter2,'Q3.Grayscale_image_results/filter2_waterplane.jpg')

In [267]:
def grayscale_to_color(image, color_filter, result_image):
    '''
    pass the filter over the pixels, set values for each channel corr. to mask 
    '''
    new_image = np.zeros((image.shape[0], image.shape[1],3))    
    for i in range(0, image.shape[0]-1, 1):
        for j in range(0, image.shape[1]-1, 1):
            r = np.where(color_filter == 0)
            g = np.where(color_filter == 1)
            b = np.where(color_filter == 2)
            
            for x,y in zip(r[0],r[1]):
                new_image[i][j][2] = image[i+x][j+y] # red component
            
            for x,y in zip(g[0],g[1]):
                new_image[i][j][1] += image[i+x][j+y]
            new_image[i][j][1] = new_image[i][j][1] / 2
            
            for x,y in zip(b[0],b[1]):
                new_image[i][j][0] = image[i+x][j+y] # blue
            
            color_filter = np.roll(color_filter, shift=1, axis=1)
        
        color_filter = np.roll(color_filter, shift=1, axis=1)
        color_filter = np.roll(color_filter, shift=1, axis=0)
    cv.imwrite(result_image,new_image)

In [271]:
color_filter1 = np.array([[1,2],[0,1]]) # Bayer's mask which Canon uses
color_filter2 = np.array([[2,1],[1,0]]) 

image1 = cv.imread('Q3.Grayscale_image_results/filter1_fall_colors.jpg',0)
image2 = cv.imread('Q3.Grayscale_image_results/filter2_fall_colors.jpg',0)
grayscale_to_color(image1, color_filter1,'Q3.Demosaicking_results/filter1_fall_colors.png')
grayscale_to_color(image2, color_filter2,'Q3.Demosaicking_results/filter2_fall_colors.png')

image1 = cv.imread('Q3.Grayscale_image_results/filter1_orange_flower.jpg',0)
image2 = cv.imread('Q3.Grayscale_image_results/filter2_orange_flower.jpg',0)
grayscale_to_color(image1, color_filter1,'Q3.Demosaicking_results/filter1_orange_flower.png')
grayscale_to_color(image2, color_filter2,'Q3.Demosaicking_results/filter2_orange_flower.png')

image1 = cv.imread('Q3.Grayscale_image_results/filter1_waterplane.jpg',0)
image2 = cv.imread('Q3.Grayscale_image_results/filter2_waterplane.jpg',0)
grayscale_to_color(image1, color_filter1,'Q3.Demosaicking_results/filter1_waterplane.png')
grayscale_to_color(image2, color_filter2,'Q3.Demosaicking_results/filter2_waterplane.png')